<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import csv
import pandas as pd

url = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
sales = pd.read_csv(url)

print(sales.count())

#
#uniq = set()
#for i, row in sales.iterrows():
#  uniq.add(row["Product_Category"])
#
#print("Produktových kategorií:"+ str(len(uniq)) )  # vypíše: Států/lokací:32
#print(uniq)  # vypíše např.: {'Romania', 'Luxembourg', 'France', 'Czechia',...}

CustomerID             52924
Transaction_ID         52924
Transaction_Date       52924
Product_SKU            52924
Product_Description    52924
Product_Category       52924
Quantity               52924
Avg_Price              52924
Delivery_Charges       52924
Coupon_Status          52924
dtype: int64
